In [85]:
import numpy as np
import numpy.fft as fft
import matplotlib.pyplot as plt
from scipy import signal
from IPython import display
from PIL import Image
import math

%matplotlib inline

In [91]:
# number of points
n = 5

# range of points
r = 50

# number of images
m = 3

In [92]:
np.random.seed(805764)

lst = list(range(0, r + 1))

# psf
h = np.sort(np.hstack((np.random.choice(lst, (n, 1)), np.random.choice(lst, (n, 1)))), axis=0)

# each image
v = np.empty((m, n, 2))
for i in range(m):
    v[i] = np.sort(np.hstack((np.random.choice(lst, (n, 1)), np.random.choice(lst, (n, 1)))), axis=0)

In [93]:
# calculated convolutions
b = np.empty((m, 2, n, n))

# Each x, y dimension
for i in range(2):
    # Each point of (x, y) in h
    for j in range(n):
        # Each point of (x, y) in v
        for k in range(n):
            for l in range(m):
                b[l, i, j, k] = h[j, i] + v[l, k, i]

# Each b is a (x, y) list of points of where it's 1
b_temp = np.empty((m, n * n, 2))
for i in range(m):
    b_temp[i] = np.hstack((np.reshape(b[i][0], (b[i].shape[1] * b[i].shape[2], 1)), np.reshape(b[i][1], (b[i].shape[1] * b[i].shape[2], 1))))
    
b = b_temp

In [94]:
A1 = np.eye(n).repeat(n, axis = 0).astype('uint8')
A2 = np.tile(np.eye(n), (n, 1)).astype('uint8')
A3 = np.zeros((1, 2 * n)).astype('uint8')
A3[0, n] = 1
A = np.vstack((np.hstack((A1, A2)), A3))
print(A)

vec = np.random.permutation(n * n)
rness = n

[[1 0 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 1 0 0 0]
 [1 0 0 0 0 0 0 1 0 0]
 [1 0 0 0 0 0 0 0 1 0]
 [1 0 0 0 0 0 0 0 0 1]
 [0 1 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 1 0 0 0]
 [0 1 0 0 0 0 0 1 0 0]
 [0 1 0 0 0 0 0 0 1 0]
 [0 1 0 0 0 0 0 0 0 1]
 [0 0 1 0 0 1 0 0 0 0]
 [0 0 1 0 0 0 1 0 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 1]
 [0 0 0 1 0 1 0 0 0 0]
 [0 0 0 1 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 1 0 0]
 [0 0 0 1 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 1]
 [0 0 0 0 1 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0]
 [0 0 0 0 1 0 0 1 0 0]
 [0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0 0 0 0]]


In [95]:
b1 = A.dot(np.vstack((v[0], h)))
b1 = b1[:b1.shape[0] - 1,]
besterror = np.linalg.norm(np.vstack((b1[vec], np.array([[0, 0]]))) - A.dot(np.linalg.pinv(A)).dot(np.vstack((b1[vec], np.array([[0, 0]])))))
print('initial: ' + str(besterror))
for iteration in range(20):
    print('iteration ' + str(iteration))
    order = np.random.permutation(b1.shape[0])
    for i in range(b1.shape[0]):
        vectemp = vec
        swapdistance = np.round(np.random.randn(1)[0] * rness)
        swapind = int(min(max(order[i] + swapdistance, 0), b1.shape[0] - 1))
        vec[order[i]] = vectemp[swapind]
        vec[swapind] = vectemp[order[i]]
        error = np.linalg.norm(np.vstack((b1[vec], np.array([[0, 0]]))) - A.dot(np.linalg.pinv(A)).dot(np.vstack((b1[vec], np.array([[0, 0]])))))
        if error < besterror:
            besterror = error
            print('improved: ' + str(besterror))
        else:
            vec = vectemp
    if besterror < 1e-1:
        break
print('took ' + str(iteration) + ' iterations')

initial: 95.78517630614876
iteration 0
improved: 94.50079364746097
improved: 86.50271672034353
improved: 86.47635514983271
improved: 85.99581385160559
improved: 77.87682582129294
improved: 77.80899690909786
improved: 77.71486344323073
iteration 1
iteration 2
improved: 77.36821052602936
improved: 74.53482407572987
improved: 72.93997532217844
improved: 64.48751817212381
iteration 3
improved: 63.59874212592572
improved: 58.80544192504635
iteration 4
iteration 5
iteration 6
iteration 7
iteration 8
iteration 9
iteration 10
iteration 11
iteration 12
iteration 13
iteration 14
improved: 57.93996893337102
iteration 15
iteration 16
improved: 52.63610927870715
iteration 17
iteration 18
iteration 19
improved: 52.63610927870714
improved: 3.343312556551121e-13
took 19 iterations
